Navier-Stokes
This benchmark is to model the incompressible and viscous flow on a unit torus, where the fluid density is constant and viscosity is set as 10e-5 (Li et al., 2021). The fluid field is discretized into 64×64 regular grid. The task is to predict the fluid in the next 10 steps based on the observations in the past 10 steps. 1000 fluids with different initial conditions are generated for training, and 200 new samples are used for test.

In [76]:
import os
import matplotlib.pyplot as plt
import argparse
import scipy.io as scio
import numpy as np
import torch
from tqdm import *
# from utils.testloss import TestLoss
# from model_dict import get_model

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
print(f"Device Name: {torch.cuda.get_device_name(0)}")

Device Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [77]:
lr = 1e-3
epochs = 5
weight_decay = 1e-5
model = 'Transolver_2D'
n_hidden = 64
n_layers = 3
n_heads = 4
batch_size = 8
gpu = '0'
max_grad_norm = None
downsample = 1
mlp_ratio = 1
dropout = 0.0
unified_pos = 0
ref = 8
slice_num = 32
eval = 0
save_name = 'ns_2d_UniPDE'
data_path = 'data'



In [78]:
data_path = data_path + '/ns/NavierStokes_V1e-5_N1200_T20.mat'

ntrain = 1000
ntest=200
T_in=10
T=10
step=1
eval=0
save_name='ns_2d_UniPDE'

In [79]:
def count_parameters(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        total_params += params
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [80]:
r = downsample
h = int(((64 - 1) / r) + 1)
data = scio.loadmat(data_path)
print(data.keys())
print('u:',data['u'].shape)


dict_keys(['__header__', '__version__', '__globals__', 'a', 'u', 't'])
u: (1200, 64, 64, 20)


In [81]:
train_a = data['u'][:ntrain, ::r, ::r, :T_in]
train_a = train_a.reshape(train_a.shape[0], -1, train_a.shape[-1])
train_a = torch.from_numpy(train_a)

train_u = data['u'][:ntrain, ::r, ::r, T_in:T + T_in]
train_u = train_u.reshape(train_u.shape[0], -1, train_u.shape[-1])
train_u = torch.from_numpy(train_u)

test_a = data['u'][-ntest:, ::r, ::r, :T_in]
test_a = test_a.reshape(test_a.shape[0], -1, test_a.shape[-1])
test_a = torch.from_numpy(test_a)

test_u = data['u'][-ntest:, ::r, ::r, T_in:T + T_in]
test_u = test_u.reshape(test_u.shape[0], -1, test_u.shape[-1])
test_u = torch.from_numpy(test_u)


In [82]:
x = np.linspace(0, 1, h)
y = np.linspace(0, 1, h)
x, y = np.meshgrid(x, y)
pos = np.c_[x.ravel(), y.ravel()]
pos = torch.tensor(pos, dtype=torch.float).unsqueeze(0)
pos_train = pos.repeat(ntrain, 1, 1)
pos_test = pos.repeat(ntest, 1, 1)

In [83]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(pos_train, train_a, train_u),
                                               batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(pos_test, test_a, test_u),
                                              batch_size=batch_size, shuffle=False, pin_memory=True)

In [84]:
data_iter = iter(train_loader)
data_next = next(data_iter)
data_next[0].shape
data_next[1].shape
data_next[2].shape


torch.Size([8, 4096, 10])

In [85]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'PDE-Solving-StandardBenchmark'))
from model import Transolver_Irregular_Mesh, Transolver_Structured_Mesh_2D, Transolver_Structured_Mesh_3D
from utils.testloss import TestLoss

In [86]:
model = Transolver_Structured_Mesh_2D.Model(
    space_dim=2,
    n_layers=n_layers,
    n_hidden=n_hidden,
    dropout=dropout,
    n_head=n_heads,
    Time_Input=False,
    mlp_ratio=mlp_ratio,
    fun_dim=T_in,
    out_dim=1,
    slice_num=slice_num,
    ref=ref,
    unified_pos=unified_pos,
    H=h, W=h
).cuda()

In [87]:
model.modules

<bound method Module.modules of Model(
  (preprocess): MLP(
    (linear_pre): Sequential(
      (0): Linear(in_features=12, out_features=128, bias=True)
      (1): GELU(approximate='none')
    )
    (linear_post): Linear(in_features=128, out_features=64, bias=True)
    (linears): ModuleList()
  )
  (blocks): ModuleList(
    (0-1): 2 x Transolver_block(
      (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (Attn): Physics_Attention_Structured_Mesh_2D(
        (softmax): Softmax(dim=-1)
        (dropout): Dropout(p=0.0, inplace=False)
        (in_project_x): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (in_project_fx): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (in_project_slice): Linear(in_features=16, out_features=32, bias=True)
        (to_q): Linear(in_features=16, out_features=16, bias=False)
        (to_k): Linear(in_features=16, out_features=16, bias=False)
        (to_v): Linear(in_features=16, out_fea

In [88]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, epochs=epochs,
                                                    steps_per_epoch=len(train_loader))
myloss = TestLoss(size_average=False)

count_parameters(model)

Total Trainable Params: 273901


273901

In [89]:
for ep in range(epochs):
    model.train()
    train_l2_step = 0
    train_l2_full = 0

    for x, fx, yy in train_loader:
        loss = 0
        x, fx, yy = x.cuda(), fx.cuda(), yy.cuda()  # x: B,4096,2    fx: B,4096,T   y: B,4096,T
        bsz = x.shape[0] # batch size
        print('batch size:', bsz)

        for t in range(0, T, step):
            y = yy[..., t:t + step]
            im = model(x, fx=fx)  # B , 4096 , 1
            loss += myloss(im.reshape(bsz, -1), y.reshape(bsz, -1))
            if t == 0:
                pred = im
            else:
                pred = torch.cat((pred, im), -1)
            fx = torch.cat((fx[..., step:], y), dim=-1)  # detach() & groundtruth

        train_l2_step += loss.item()
        train_l2_full += myloss(pred.reshape(bsz, -1), yy.reshape(bsz, -1)).item()
        optimizer.zero_grad()
        loss.backward()
        if max_grad_norm is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
            
        test_l2_step = 0
        test_l2_full = 0

        model.eval()

        with torch.no_grad():
            for x, fx, yy in test_loader:
                loss = 0
                x, fx, yy = x.cuda(), fx.cuda(), yy.cuda()  # x : B, 4096, 2  fx : B, 4096  y : B, 4096, T
                bsz = x.shape[0]
                for t in range(0, T, step):
                    y = yy[..., t:t + step]
                    im = model(x, fx=fx)
                    loss += myloss(im.reshape(bsz, -1), y.reshape(bsz, -1))
                    if t == 0:
                        pred = im
                    else:
                        pred = torch.cat((pred, im), -1)
                    fx = torch.cat((fx[..., step:], im), dim=-1)

                test_l2_step += loss.item()
                test_l2_full += myloss(pred.reshape(bsz, -1), yy.reshape(bsz, -1)).item()

        print(
            "Epoch {} , train_step_loss:{:.5f} , train_full_loss:{:.5f} , test_step_loss:{:.5f} , test_full_loss:{:.5f}".format(
                ep, train_l2_step / ntrain / (T / step), train_l2_full / ntrain, test_l2_step / ntest / (T / step),
                    test_l2_full / ntest))
        

batch size: 8
Epoch 0 , train_step_loss:0.00814 , train_full_loss:0.00809 , test_step_loss:1.00032 , test_full_loss:1.00020
batch size: 8


KeyboardInterrupt: 

In [90]:
torch.cuda.empty_cache()